Import necessary libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from collections import Counter
import spacy

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

The dataset that will be used is all the [Proposed Amendments to the United States Constitution](https://www.archives.gov/open/dataset-amendments); the dataset uses a unique encoding which is why it isn't UTF8

In [40]:
data = pd.read_csv('all_proposed_amendments.csv', encoding='ISO-8859-1')

There may be some columns that have useless or little to no data in them, so they will be scowered through to determine which are needed.

In [41]:
for column in data.columns:
	group = data.groupby(column)
	if (len(group) < 100):
		print(f"{column} ({len(group)}): {group.groups.keys()}")

source_code (7): dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G'])
source_citation (7): dict_keys(['Ames, Herman V. The Proposed Amendments to the Constitution of the United States During the First Century of Its History, American Historical Society Annual Report for the year 1896; H. Doc. 353 part 2, 54th Cong., 2d Session, volume 3550-2, Washington: U.S. Government Printing Office, 1897.', 'Davis, Richard. Proposed Amendments to the Constitution of the United States Introduced in Congress from the 91st Congress, 1st Session through the 98th Congress, 2d Session, January 1969-December 1984, (CRS Report No. 85-36 GOV), Washington: Congressional Research Service, 1985.', 'Harris, Daryl B. Proposed Amendments to the Constitution: 99th-101st Congress (1985-1990), (CRS Report No. 92-555 GOV), Washington: Congressional Research Service, 1992.', 'Proposed Amendments to the Constitution of the United States Introduced in Congress from the 69th Congress, 2d Session through the 87th Congress, 2d S

Because the `source_code`, `date_approximation`, `last_modified` columns have almost no data, they will be dropped from the dataset. So will `identifier` since it is essentially just the index of the row

In [42]:
data.drop(columns=['identifier', 'source_code', 'date_approximation', 'last_modified'], inplace=True)

There is a lot of duplicates within these proposed amendments since they go through multiple stages within Congress and HoS and could take months, I will make a new dataset that filters out duplicates within their `title_or_description_from_source<` column and keeps the latest entry; first by tokenizing the column and filtering those duplicates.

In [43]:
# Tokenize the title_or_description_from_source column
data['title_tokens'] = data['title_or_description_from_source'].apply(
	lambda x: nltk.word_tokenize(x.lower()) if isinstance(x, str) else []
)

# Drop the duplicates within the 'title_tokens" column
data_unique = data.copy().drop_duplicates(subset='title_tokens', keep='last')

'''
To make a new csv file with the unique words:
data_unique.to_csv('all_proposed_amendments_cleaned.csv', index=False)
'''

print(data.shape)
print(data_unique.shape)
print(f'Dropped {data.shape[0] - data_unique.shape[0]} duplicate entries based on title_tokens.')
data_unique.head()

(11797, 14)
(4627, 14)
Dropped 7170 duplicate entries based on title_tokens.


,source_citation,source_index_number,title_or_description_from_source,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,title_tokens
7,"Ames, Herman V. The Proposed Amendments to the...",8,Trial by jury in civil action,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, action]"
20,"Ames, Herman V. The Proposed Amendments to the...",21,Trial by jury in civil cases,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, cases]"
23,"Ames, Herman V. The Proposed Amendments to the...",24,Religion,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[religion]
29,"Ames, Herman V. The Proposed Amendments to the...",30,Publication of journals annually,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[publication, of, journals, annually]"
34,"Ames, Herman V. The Proposed Amendments to the...",35,Period of enlistment of soldiers limited,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[period, of, enlistment, of, soldiers, limited]"


In [44]:
def preprocess_text_list(data):
    processed_list = []
    for text in data:
        # Tokenize into sentences
        tokens = word_tokenize(str(text).lower())
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
        processed_list.append(" ".join(tokens))
    return processed_list
processed_texts = preprocess_text_list(data["title_or_description_from_source"])
print(processed_texts[:])

['reservation nondelegated powers', 'apportionment representatives', 'restriction federal control election senators representatives', 'restriction upon levying direct taxes', 'commercial monopolies prohibited', 'indictment grand jury', 'jurisdiction federal courts', 'trial jury civil action', 'titles nobility', 'restriction federal control election senators representatives', 'reservation nondelegated powers', 'restriction upon levying direct taxes', 'oath', 'reservation nondelegated powers', 'apportionment representatives', 'restriction federal control election senators representatives', 'restriction upon levying direct taxes', 'commercial monopolies prohibited', 'indictment grand jury', 'jurisdiction federal courts', 'trial jury civil cases', 'titles nobility', 'standing army time peace', 'religion', 'right bear arms', 'reservation nondelegated powers', 'apportionment representatives', 'restriction upon levying direct taxes', 'senators representatives ineligible civil office term', 'p

In [45]:
wordsplit = [word for processed_texts in processed_texts for word in processed_texts.split()]
print(wordsplit)

['reservation', 'nondelegated', 'powers', 'apportionment', 'representatives', 'restriction', 'federal', 'control', 'election', 'senators', 'representatives', 'restriction', 'upon', 'levying', 'direct', 'taxes', 'commercial', 'monopolies', 'prohibited', 'indictment', 'grand', 'jury', 'jurisdiction', 'federal', 'courts', 'trial', 'jury', 'civil', 'action', 'titles', 'nobility', 'restriction', 'federal', 'control', 'election', 'senators', 'representatives', 'reservation', 'nondelegated', 'powers', 'restriction', 'upon', 'levying', 'direct', 'taxes', 'oath', 'reservation', 'nondelegated', 'powers', 'apportionment', 'representatives', 'restriction', 'federal', 'control', 'election', 'senators', 'representatives', 'restriction', 'upon', 'levying', 'direct', 'taxes', 'commercial', 'monopolies', 'prohibited', 'indictment', 'grand', 'jury', 'jurisdiction', 'federal', 'courts', 'trial', 'jury', 'civil', 'cases', 'titles', 'nobility', 'standing', 'army', 'time', 'peace', 'religion', 'right', 'bea

In [46]:
freq2 = Counter(wordsplit)
freq2.most_common(50)

[('president', 2640),
 ('states', 2342),
 ('united', 1840),
 ('amendment', 1485),
 ('congress', 1417),
 ('constitution', 1380),
 ('rights', 1334),
 ('election', 1279),
 ('equal', 1183),
 ('vice', 1175),
 ('proposing', 1145),
 ('public', 873),
 ('representatives', 855),
 ('vote', 762),
 ('right', 758),
 ('women', 755),
 ('men', 732),
 ('office', 720),
 ('term', 699),
 ('schools', 695),
 ('providing', 684),
 ('prayer', 674),
 ('budget', 658),
 ('state', 643),
 ('popular', 507),
 ('members', 495),
 ('senators', 474),
 ('federal', 469),
 ('terms', 461),
 ('years', 443),
 ('balancing', 435),
 ('sex', 418),
 ('regardless', 414),
 ('relating', 386),
 ('court', 382),
 ('power', 379),
 ('judges', 373),
 ('shall', 373),
 ('apportionment', 369),
 ('relative', 368),
 ('prohibit', 360),
 ('supreme', 358),
 ('house', 350),
 ('direct', 312),
 ('proposal', 310),
 ('choice', 308),
 ('veto', 299),
 ('provide', 281),
 ('district', 276),
 ('limit', 270)]

In [47]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])
doc = nlp(" ".join(wordsplit))

In [48]:
nlp = spacy.load("en_core_web_sm")
lemmas = [token.lemma_ for token in doc]
print(lemmas)

['reservation', 'nondelegate', 'power', 'apportionment', 'representative', 'restriction', 'federal', 'control', 'election', 'senator', 'representative', 'restriction', 'upon', 'levy', 'direct', 'taxis', 'commercial', 'monopoly', 'prohibit', 'indictment', 'grand', 'jury', 'jurisdiction', 'federal', 'court', 'trial', 'jury', 'civil', 'action', 'title', 'nobility', 'restriction', 'federal', 'control', 'election', 'senator', 'representative', 'reservation', 'nondelegate', 'power', 'restriction', 'upon', 'levy', 'direct', 'taxis', 'oath', 'reservation', 'nondelegate', 'power', 'apportionment', 'representative', 'restriction', 'federal', 'control', 'election', 'senator', 'representative', 'restriction', 'upon', 'levy', 'direct', 'taxis', 'commercial', 'monopoly', 'prohibit', 'indictment', 'grand', 'jury', 'jurisdiction', 'federal', 'court', 'trial', 'jury', 'civil', 'case', 'title', 'nobility', 'stand', 'army', 'time', 'peace', 'religion', 'right', 'bear', 'arm', 'reservation', 'nondelegate'

In [49]:
freq3 = Counter(lemmas)
freq3.most_common(50)

[('president', 2648),
 ('right', 2090),
 ('states', 1872),
 ('united', 1833),
 ('amendment', 1609),
 ('election', 1417),
 ('congress', 1417),
 ('constitution', 1380),
 ('propose', 1232),
 ('equal', 1183),
 ('vice', 1175),
 ('term', 1157),
 ('state', 1114),
 ('provide', 1039),
 ('public', 873),
 ('woman', 847),
 ('representative', 841),
 ('vote', 820),
 ('school', 787),
 ('office', 767),
 ('man', 734),
 ('prayer', 721),
 ('budget', 662),
 ('prohibit', 607),
 ('year', 557),
 ('member', 531),
 ('senator', 510),
 ('popular', 507),
 ('court', 489),
 ('power', 482),
 ('federal', 469),
 ('limit', 457),
 ('balance', 422),
 ('sex', 418),
 ('regardless', 414),
 ('relate', 399),
 ('house', 394),
 ('district', 385),
 ('shall', 373),
 ('apportionment', 369),
 ('relative', 369),
 ('judge', 366),
 ('supreme', 358),
 ('direct', 312),
 ('item', 311),
 ('proposal', 311),
 ('veto', 309),
 ('choice', 308),
 ('tax', 302),
 ('appropriation', 288)]

In [50]:
freq3 = pd.DataFrame(freq3.most_common(50), columns=['word', 'frequency'])
freq3

,word,frequency
0,president,2648
1,right,2090
2,states,1872
3,united,1833
4,amendment,1609
5,election,1417
6,congress,1417
7,constitution,1380
8,propose,1232
9,equal,1183


In [51]:
def most_common_year_from_df(word):
    # Filter rows where the word appears in the text
    mask = data['title_or_description_from_source'].str.lower().str.contains(r'\b{}\b'.format(word), na=False)
    years = data.loc[mask, 'year']
    if not years.empty:
        return years.value_counts().idxmax()
    return None

freq3["Year mostly appear"] = freq3["word"].apply(most_common_year_from_df)
freq3

,word,frequency,Year mostly appear
0,president,2648,1969.0
1,right,2090,1969.0
2,states,1872,1993.0
3,united,1833,1993.0
4,amendment,1609,1993.0
5,election,1417,1969.0
6,congress,1417,1923.0
7,constitution,1380,1993.0
8,propose,1232,1861.0
9,equal,1183,1969.0


In [52]:
number_documents = len(data)
print(number_documents)

11797


In [53]:
number_of_documents_with_document = sum(1 for doc in processed_texts if 'president' in word_tokenize(doc))
print("Number of documents with the word president:", number_of_documents_with_document)

Number of documents with the word president: 1479


In [54]:
idf = number_documents/number_of_documents_with_document
print("Inverse Document Frequency for 'president':", idf)

Inverse Document Frequency for 'president': 7.976335361730899


In [55]:
tf = [word_tokenize(doc).count('president') for doc in processed_texts]
print(tf)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [56]:
tf_idf = []
for i, doc in enumerate(processed_texts):
    tf_i = word_tokenize(doc).count('president')
    doc_length = len(word_tokenize(doc))
    if doc_length > 0:
        tf_normalized = tf_i / doc_length
    else:
        tf_normalized = 0
    tfidf_value = tf_normalized * idf
    tf_idf.append(tfidf_value)
    print(f"President {i}: TF = {tf_i}, Doc Length = {doc_length}, TF_norm = {tf_normalized:.2f}, IDF = {idf:.2f}, TF-IDF = {tfidf_value:.4f}")

President 0: TF = 0, Doc Length = 3, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 1: TF = 0, Doc Length = 2, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 2: TF = 0, Doc Length = 6, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 3: TF = 0, Doc Length = 5, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 4: TF = 0, Doc Length = 3, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 5: TF = 0, Doc Length = 3, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 6: TF = 0, Doc Length = 3, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 7: TF = 0, Doc Length = 4, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 8: TF = 0, Doc Length = 2, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 9: TF = 0, Doc Length = 6, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 10: TF = 0, Doc Length = 3, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 11: TF = 0, Doc Length = 5, TF_norm = 0.00, IDF = 7.98, TF-IDF = 0.0000
President 12: TF = 0, Doc 

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
stop = stopwords.words('english')
tfidfvec = TfidfVectorizer(min_df=2,lowercase=True,stop_words=stop)
# min_df=2 means we only consider words that appear in at least 2 documents

tfidf_bow = tfidfvec.fit_transform(processed_texts)
print(tfidf_bow.toarray())
print(tfidf_bow.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(11797, 1637)


In [58]:
#It worked without using the ', index=processed_texts.index' at the end.
tfidf = pd.DataFrame(tfidf_bow.toarray(), columns=tfidfvec.get_feature_names_out())
tfidf.head(5)

,abode,abolish,abolished,abolishes,abolishing,abolishment,abolition,abortion,abortions,abridge,...,work,world,would,writing,writs,xi,xiv,year,years,yeas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
tfidf.max().sort_values(ascending=False).head(50)

used              1.000000
requisitions      1.000000
reconstruction    1.000000
religion          1.000000
amendment         1.000000
shall             1.000000
ratification      1.000000
prohibited        1.000000
right             1.000000
suffrage          1.000000
special           1.000000
liberty           1.000000
nan               1.000000
oath              1.000000
power             1.000000
prevention        1.000000
departments       1.000000
choice            1.000000
abolish           1.000000
abolished         1.000000
impeachment       1.000000
franchise         1.000000
balancing         1.000000
internal          1.000000
jurisdiction      1.000000
judges            1.000000
initiative        1.000000
busing            1.000000
citizenship       1.000000
clause            1.000000
compensation      1.000000
pres              0.950662
farmers           0.945788
adjournment       0.943225
equality          0.938312
repeals           0.938027
indians           0.937516
s

In [60]:
print(type(wordsplit))
text_set = set(wordsplit)
print(text_set)

<class 'list'>
{'beer', 'defray', 'special', 'extraordinary', 'terminate', 'concurrent', 'marketing', 'prohibitions', 'reading', 'expenses', 'rentention', 'daily', 'southern', 'appointments', 'nationwide', 'interstate', 'others', 'regulated', 'back', 'guam', 'offset', 'renomination', 'saturday', 'resident', 'starting', 'laying', 'tribes', 'federally', 'stripes', 'hamilton', 'qualified', 'applied', 'reconfirmation', 'imprisoned', 'released', 'uniform', 'systems', 'racial', 'saviour', 'month', 'parent', 'clause', 'tie', 'previous', 'prohibition', 'received', 'protective', 'thereto', 'treaty', 'outstanding', 'v', 'census', 'freeing', 'relation', 'slavery', 'matter', 'operations', 'decency', 'vocation', 'choose', 'per', 'village', 'whenever', 'encourage', 'invalid', 'allows', 'inhabitants', 'basis', 'requirements', 'mandated', 'white', 'proportionally', 'remarry', 'presidential', 'fixing', 'water', 'secessionist', 'magistrates', 'felons', 'amending', 'restoring', 'level', 'earlier', 'color

In [61]:
data_unique.head()

,source_citation,source_index_number,title_or_description_from_source,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,title_tokens
7,"Ames, Herman V. The Proposed Amendments to the...",8,Trial by jury in civil action,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, action]"
20,"Ames, Herman V. The Proposed Amendments to the...",21,Trial by jury in civil cases,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, cases]"
23,"Ames, Herman V. The Proposed Amendments to the...",24,Religion,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[religion]
29,"Ames, Herman V. The Proposed Amendments to the...",30,Publication of journals annually,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[publication, of, journals, annually]"
34,"Ames, Herman V. The Proposed Amendments to the...",35,Period of enlistment of soldiers limited,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[period, of, enlistment, of, soldiers, limited]"


<h1> Prediction: Number of Amendment Proposal w/ factors like Decades, Politcal Party in Power at the time, and whether the country was in war

In [64]:
data_unique['decades'] = (data_unique['year'] // 10) * 10
data_unique.head()

,source_citation,source_index_number,title_or_description_from_source,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,title_tokens,decades
7,"Ames, Herman V. The Proposed Amendments to the...",8,Trial by jury in civil action,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, action]",1780.0
20,"Ames, Herman V. The Proposed Amendments to the...",21,Trial by jury in civil cases,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[trial, by, jury, in, civil, cases]",1780.0
23,"Ames, Herman V. The Proposed Amendments to the...",24,Religion,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[religion],1780.0
29,"Ames, Herman V. The Proposed Amendments to the...",30,Publication of journals annually,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[publication, of, journals, annually]",1780.0
34,"Ames, Herman V. The Proposed Amendments to the...",35,Period of enlistment of soldiers limited,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[period, of, enlistment, of, soldiers, limited]",1780.0


In [65]:
counts_df = data_unique.groupby('decades').count()
counts_df = counts_df.reset_index()
counts_df

,decades,source_citation,source_index_number,title_or_description_from_source,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,title_tokens
0,1780.0,173,174,174,174,84,84,3,3,0,0,31,28,0,174
1,1790.0,20,20,20,20,14,14,11,11,0,0,4,3,0,20
2,1800.0,37,37,37,37,27,27,26,26,0,0,22,22,0,37
3,1810.0,40,40,40,40,18,18,17,17,0,0,17,14,0,40
4,1820.0,75,76,76,76,62,62,55,55,0,0,57,55,0,76
5,1830.0,80,81,81,81,39,39,39,39,0,0,35,31,0,81
6,1840.0,44,44,44,44,39,39,38,38,0,0,38,38,0,44
7,1850.0,15,15,15,15,12,12,12,12,0,0,12,12,0,15
8,1860.0,409,409,409,409,196,196,191,191,0,0,192,182,5,409
9,1870.0,138,138,138,138,122,122,122,122,0,0,122,118,104,138


In [66]:
# create new df with only necessary columns we will be using
decades_df = pd.DataFrame({
    'decades' : counts_df['decades'],
    'proposal_counts' : counts_df['title_or_description_from_source']
    
})
decades_df

,decades,proposal_counts
0,1780.0,174
1,1790.0,20
2,1800.0,37
3,1810.0,40
4,1820.0,76
5,1830.0,81
6,1840.0,44
7,1850.0,15
8,1860.0,409
9,1870.0,138


In [69]:
# create new dataset for external information on war & political power
political_power = pd.read_excel('pol_power.xlsx')
political_power['decades'] = political_power['decades'].astype(float)
political_power.head()

,decades,in war?,"politcal party in power (democratic, republican, neither, or both)"
0,1780.0,1,neither
1,1790.0,1,neither
2,1800.0,1,both
3,1810.0,1,both
4,1820.0,0,both


In [70]:
# add new info into our existing df
decades_df['in_war'] = political_power['in war?']
decades_df['politcal_power'] = political_power['politcal party in power (democratic, republican, neither, or both)']
decades_df

,decades,proposal_counts,in_war,politcal_power
0,1780.0,174,1,neither
1,1790.0,20,1,neither
2,1800.0,37,1,both
3,1810.0,40,1,both
4,1820.0,76,0,both
5,1830.0,81,1,democratic
6,1840.0,44,1,democratic
7,1850.0,15,1,democratic
8,1860.0,409,1,republican
9,1870.0,138,1,republican


In [71]:
# one hot encode politcal party for modeling
df_encoded = pd.get_dummies(decades_df, columns=['politcal_power'])
df_encoded = df_encoded.drop(['politcal_power_both'], axis = 1) # to avoid multicollinearity
df_encoded

,decades,proposal_counts,in_war,politcal_power_democratic,politcal_power_neither,politcal_power_republican
0,1780.0,174,1,False,True,False
1,1790.0,20,1,False,True,False
2,1800.0,37,1,False,False,False
3,1810.0,40,1,False,False,False
4,1820.0,76,0,False,False,False
5,1830.0,81,1,True,False,False
6,1840.0,44,1,True,False,False
7,1850.0,15,1,True,False,False
8,1860.0,409,1,False,False,True
9,1870.0,138,1,False,False,True


In [72]:
X = df_encoded.drop(columns=['proposal_counts', 'decades'])  # predictors
y = df_encoded['proposal_counts'] # target

In [73]:
model = LinearRegression()
model.fit(X, y)

# View coefficients
coefficients = pd.Series(model.coef_, index=X.columns)
print("Intercept:", model.intercept_)
print("Coefficients:")
print(coefficients)

Intercept: 187.81679389312987
Coefficients:
in_war                      -52.755725
politcal_power_democratic    60.521628
politcal_power_neither      -38.061069
politcal_power_republican    72.464801
dtype: float64


In [74]:
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 16014.509471303363
R-squared: 0.1079064955632314


In [75]:
import numpy as np
avg_prop_count = df_encoded['proposal_counts'].mean()
RMSE = np.sqrt(mse)

print(avg_prop_count, RMSE)

192.75 126.54844713114169


In [76]:
relative_error = RMSE/avg_prop_count
print(relative_error)

0.656541878760787


In [77]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.glm(formula='proposal_counts ~ in_war + politcal_power_democratic + politcal_power_neither + politcal_power_republican',
                data=df_encoded, family=sm.families.Poisson()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        proposal_counts   No. Observations:                   24
Model:                            GLM   Df Residuals:                       19
Model Family:                 Poisson   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1146.9
Date:                Tue, 01 Jul 2025   Deviance:                       2131.9
Time:                        13:35:25   Pearson chi2:                 2.17e+03
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [78]:
democrat = np.exp(0.3104)
neither = np.exp(-0.4041)
republican = np.exp(0.3563)
war = np.exp(-0.2346)

print(democrat, neither, republican, war)

1.3639705932663844 0.6675773601949113 1.4280358947770884 0.7908871412226887


Given the Poisson Regression summary, we can conclude the following:
1. proposal count is 36% higher in democratic eras compared to when both parties are in power
2. proposal count is 33% lower when neither parties are in power
3. proposal counts is 43% higher in republican eras compared to when both parties are in power
4. during war periods, proposal counts are 21% lower on average

all of the coefficients have very low p-values (P>|z| = 0.000), meaning they’re statistically significant at any standard threshold (0.05, 0.01, etc). this means that we can rely on the interpretations above to make a prediction on amendment proposal counts based on politcal party in power and war status of the country at the time.